In [1]:
import pandas as pd
import torch

In [10]:
a = torch.Tensor([[1,2,3], [2,3,45], [6,8,9], [1,1,1], [1,2,1]])
torch.topk(a, 1, dim=1)

torch.return_types.topk(
values=tensor([[ 3.],
        [45.],
        [ 9.],
        [ 1.],
        [ 2.]]),
indices=tensor([[2],
        [2],
        [2],
        [0],
        [1]]))

In [12]:
for i in a.numpy():
    print(i)

[1. 2. 3.]
[ 2.  3. 45.]
[6. 8. 9.]
[1. 1. 1.]
[1. 2. 1.]


In [14]:

learning_rate = 0.0001
batch_size = 256
#embed_dim = 256
#factor_dim = 64

if torch.cuda.is_available():
    device = torch.device('cuda')
    FloatTensor = torch.cuda.FloatTensor
else:
    device = torch.device('cpu')
    FloatTensor = torch.FloatTensor
manualSeed = 706
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print('CUDA Available:', torch.cuda.is_available())

# Prepare data
user_matrix, item_matrix, train_u, train_i, train_r, neg_candidates, u_cnt, user_rating_max = data_utils.load_train_data(args.data)
test_users, test_items = data_utils.load_test_ml1m()
eval_batch_size = 100 * 151
n_users, n_items = user_matrix.shape[0], user_matrix.shape[1]

# trian_u - np array with user ids from original 



user_array = user_matrix.toarray()
item_array = item_matrix.toarray()
user_idxlist, item_idxlist = list(range(n_users)), list(range(n_items))

# Model
model = JNCF(DF_layers, DI_layers, n_users, n_items, 'concat').to(device)  # 'multi' or 'concat'

# Optimize
pair_loss_function = TOP1
point_loss_function = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr)

best_hr = 0.0
for epoch in range(args.epoch):
    model.train()

    idxlist = np.array(range(len(train_u)))
    np.random.shuffle(idxlist)
    epoch_loss, epoch_pair_loss, epoch_point_loss, epoch_i_point_loss, epoch_j_point_loss = .0, .0, .0, .0, .0

    start_time = time.time()
    for batch_idx, start_idx in enumerate(range(0, len(idxlist), batch_size)):
        end_idx = min(start_idx + batch_size, len(idxlist))
        idx = idxlist[start_idx:end_idx]

        u_ids = train_u.take(idx)
        i_ids = train_i.take(idx)
        i_ratings = train_r.take(idx)
        
        users = FloatTensor(user_array.take(u_ids, axis=0))
        items = FloatTensor(item_array.take(i_ids, axis=0))
        labels = FloatTensor(i_ratings)

        rating_max = FloatTensor(user_rating_max.take(u_ids, axis=0))
        Y_ui = labels / rating_max  # for Normalized BCE
        Y_uj = torch.zeros_like(Y_ui)  # for Negative samples point-wise loss

        # Negative Sampling
        neg_items_list = []
        for _ in range(0, n_negs):
            neg_items = one_negative_sampling(u_ids, neg_candidates)
            neg_items_list.append(neg_items)

        for neg_idx in range(0, n_negs):
            optimizer.zero_grad()
            point_loss, pair_loss = 0., 0.

            neg_ids = neg_items_list[neg_idx]
            items_j = FloatTensor(item_array.take(neg_ids, axis=0))

            y_i, y_j = model(users, items, items_j)

            i_point_loss = point_loss_function(y_i, Y_ui)  # positive items i
            j_point_loss = point_loss_function(y_j, Y_uj)  # negative items j
            point_loss = i_point_loss + j_point_loss
            pair_loss = pair_loss_function(y_i, y_j, n_negs)

            loss = args.alpha * pair_loss + (1 - args.alpha) * point_loss

            epoch_loss += loss.item()
            epoch_pair_loss += pair_loss.item()
            epoch_point_loss += point_loss.item()

            loss.backward()
            optimizer.step()
    train_time = time.time() - start_time

    # Evaluate
    model.eval()
    HR, NDCG = [], []

    time_E = time.time()
    for start_idx in range(0, len(test_users), eval_batch_size):
        end_idx = min(start_idx + eval_batch_size, len(test_users))
        u_ids = test_users[start_idx:end_idx]
        i_ids = test_items[start_idx:end_idx]

        users = FloatTensor(user_array.take(u_ids, axis=0))
        items = FloatTensor(item_array.take(i_ids, axis=0))

        preds, _ = model(users, items, items)

        e_batch_size = eval_batch_size // 100  # faster eval
        preds = torch.chunk(preds.detach().cpu(), e_batch_size)
        chunked_items = torch.chunk(torch.IntTensor(i_ids), e_batch_size)

        for i, pred in enumerate(preds):
            _, indices = torch.topk(pred, 10)
            recommends = torch.take(chunked_items[i], indices).numpy().tolist()

            gt_item = chunked_items[i][0].item()
            HR.append(hit(gt_item, recommends))
            NDCG.append(ndcg(gt_item, recommends))

    eval_time = time.time() - time_E
    #if epoch % 10 == 0:
    e_loss = epoch_loss / (batch_idx + 1)
    e_pair = epoch_pair_loss / (batch_idx + 1)
    e_point = epoch_point_loss / (batch_idx + 1)
    e_i_point = epoch_i_point_loss / (batch_idx + 1)
    e_j_point = epoch_j_point_loss / (batch_idx + 1)
    text_1 = '[Epoch {:03d}]'.format(epoch) + '\ttrain: ' + time.strftime('%M: %S', time.gmtime(train_time)) + '\tHR: {:.4f}\tNDCG: {:.4f}\n'.format(np.mean(HR), np.mean(NDCG))
    text_2 = 'Loss: {:.6f}\tPair: {:.4f}\tPoint: {:.4f}\ti_point: {:.4f}\tj_point: {:.4f}\n'.format(e_loss, e_pair, e_point, e_i_point, e_j_point)
    print(text_1[:-1])
    print(text_2[:-1])
    output.write(text_1)
    output.write(text_2)

    if np.mean(HR) > best_hr:
        best_hr, best_ndcg, best_epoch = np.mean(HR), np.mean(NDCG), epoch
result = 'DF: {} DI: {}. Best epoch {:02d}: HR = {:.4f}, NDCG = {:.4f}\n'.format(DF_layers, DI_layers, best_epoch, best_hr, best_ndcg)
print(result[:-1])
output.write(result)
output.close()
© 2022 GitHub, Inc.
Terms
Privacy
Security
Status
Docs
Contact GitHub
Pricing
API
Training
Blog
About
Loading complete

ImportError: libgfortran.so.4: cannot open shared object file: No such file or directory

In [2]:
data_path = "/home/mmarzec12/data/"

In [3]:
abc = pd.read_csv(data_path+"ratings_chunk_1.csv")

In [8]:
abc.head()

,Unnamed: 0,user_name,game_id,score,timestamp,stat_own,stat_preordered,stat_wishlist,stat_fortrade,stat_wanttoplay,stat_prevowned,stat_want,stat_wanttobuy
0,0,Lil Red Head,1,7.0,2009-03-19 21:27:53,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,hsjx945,1,8.0,2019-08-27 10:20:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,RuffRyder,1,0.0,2019-07-01 16:38:47,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,SkywalterDBZ,1,7.0,2007-06-15 14:16:50,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Olmestig,1,8.0,2019-08-25 12:46:35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
abc.shape

(10004054, 13)

In [9]:
abc.columns

Index(['Unnamed: 0', 'user_name', 'game_id', 'score', 'timestamp', 'stat_own',
       'stat_preordered', 'stat_wishlist', 'stat_fortrade', 'stat_wanttoplay',
       'stat_prevowned', 'stat_want', 'stat_wanttobuy'],
      dtype='object')